In [23]:
import os
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def calculate_similarity(file_contents):
    # Inicializar el vectorizador de frecuencia de palabras
    vectorizer = CountVectorizer()

    # Crear la matriz de términos-frecuencia
    tf_matrix = vectorizer.fit_transform(file_contents)

    # Calcular la similitud coseno entre los pares de documentos
    similarity_matrix = cosine_similarity(tf_matrix, tf_matrix)

    return similarity_matrix

def process_files_in_folder(folder_path):
    file_contents = []
    file_paths = []

    # Iterar sobre los archivos en la carpeta
    for root, dirs, files in os.walk(folder_path):
        for file_name in files:
            if file_name.endswith('.java'):
                file_path = os.path.join(root, file_name)
                file_paths.append(file_path)
                with open(file_path, 'r') as file:
                    file_contents.append(file.read())

    # Calcular la similitud entre los archivos
    similarity_matrix = calculate_similarity(file_contents)

    # Imprimir la similitud entre los archivos
    print("Similitud entre archivos:")
    for i in range(len(file_paths)):
        for j in range(i+1, len(file_paths)):
            similarity = similarity_matrix[i][j]
            if similarity > 0.9:
                print(f"{file_paths[i]} - {file_paths[j]}: {round(similarity, 2)}")

# Ruta de la carpeta donde se encuentran los archivos .java
folder_path = ".\\conplag\\minor"
process_files_in_folder(folder_path)


Similitud entre archivos:
.\conplag\minor\115c99cb.java - .\conplag\minor\3e6def38.java: 0.99
.\conplag\minor\115c99cb.java - .\conplag\minor\548ffb07.java: 1.0
.\conplag\minor\115c99cb.java - .\conplag\minor\921b6e4a.java: 1.0
.\conplag\minor\3e6def38.java - .\conplag\minor\548ffb07.java: 0.99
.\conplag\minor\3e6def38.java - .\conplag\minor\921b6e4a.java: 0.99
.\conplag\minor\548ffb07.java - .\conplag\minor\921b6e4a.java: 1.0


## Carga de Archivos con pretratados

In [24]:
import os
import re

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

     # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.java')]

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    print("Contenido limpio de", archivo, ":\n", contenido_limpio)


Contenido limpio de 115c99cb.java :
 
public class D_Java {
public static final int MOD = 998244353;
public static int mul(int a, int b) {
return (int)((long)a * (long)b % MOD);
}
int[] f;
int[] rf;
public int C(int n, int k) {
return (k < 0 || k > n) ? 0 : mul(f[n], mul(rf[n-k], rf[k]));
}
public static int pow(int a, int n) {
int res = 1;
while (n != 0) {
if ((n & 1) == 1) {
res = mul(res, a);
}
a = mul(a, a);
n >>= 1;
}
return res;
}
public static int inv(int a) {
return pow(a, MOD-2);
}
public void doIt() throws IOException {
BufferedReader in = new BufferedReader(new InputStreamReader(System.in));
StringTokenizer tok = new StringTokenizer(in.readLine());
int n = Integer.parseInt(tok.nextToken());
int k = Integer.parseInt(tok.nextToken());
f = new int[n+42];
rf = new int[n+42];
f[0] = rf[0] = 1;
for (int i = 1; i < f.length; ++i) {
f[i] = mul(f[i-1], i);
rf[i] = mul(rf[i-1], inv(i));
}
int[] events = new int[2*n];
for (int i = 0; i < n; ++i) {
tok = new StringTokenizer(in.readLine(

### Checkpoint

In [25]:
import os
import re
import javalang

def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría y saltos de línea
    contenido = re.sub(r'\n\s*', '\n', contenido)

    # Eliminar comentarios de una línea
    contenido = re.sub(r'//.*', '', contenido)

    # Eliminar comentarios de múltiples líneas
    contenido = re.sub(r'/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido

carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(carpeta) if archivo.endswith('.java')]

ast_trees = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        ast_trees[archivo] = ast_tree  # Guardar el AST en un diccionario con el nombre del archivo como clave
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)


# Comparación de subestructuras comunes entre los árboles AST almacenados en un diccionario
def comparar_subestructuras_comunes(ast_trees):
    # Diccionario para almacenar las subestructuras comunes encontradas en los árboles
    subestructuras_comunes = {}

    # Recorrer cada par de árboles AST para compararlos
    for nombre_archivo1, arbol1 in ast_trees.items():
        for nombre_archivo2, arbol2 in ast_trees.items():
            # Evitar comparar un árbol consigo mismo
            if nombre_archivo1 != nombre_archivo2:
                # Comparar los árboles AST y encontrar las subestructuras comunes
                subestructuras = encontrar_subestructuras_comunes(arbol1, arbol2)

                # Guardar las subestructuras comunes en el diccionario
                subestructuras_comunes[(nombre_archivo1, nombre_archivo2)] = subestructuras

    return subestructuras_comunes

# Función para encontrar subestructuras comunes entre dos árboles AST
def encontrar_subestructuras_comunes(arbol1, arbol2):
    # Función auxiliar para recorrer los árboles AST recursivamente
    def recorrer_arbol(nodo1, nodo2):
        # Si alguno de los nodos es None, no hay subestructuras comunes aquí
        if nodo1 is None or nodo2 is None:
            return []

        # Si los tipos de los nodos son diferentes, no hay subestructuras comunes aquí
        if type(nodo1) != type(nodo2):
            return []

        # Si los tipos de los nodos son iguales, verifica si son del mismo tipo
        if isinstance(nodo1, javalang.ast.Node) and isinstance(nodo2, javalang.ast.Node):
            # Si los nodos son del mismo tipo, compara sus subnodos recursivamente
            subestructuras_comunes = []
            for subnodo1, subnodo2 in zip(nodo1.children, nodo2.children):
                subestructuras_comunes.extend(recorrer_arbol(subnodo1, subnodo2))
            return subestructuras_comunes

        # Si los nodos son de tipo primitivo (como str, int, etc.), compara sus valores
        if nodo1 == nodo2:
            # Si los valores son iguales, se ha encontrado una subestructura común
            return [(nodo1, nodo2)]
        else:
            # Si los valores son diferentes, no hay subestructuras comunes aquí
            return []

    # Llama a la función auxiliar para iniciar el recorrido recursivo de los árboles
    return recorrer_arbol(arbol1, arbol2)


# Llamada a la función para comparar subestructuras comunes entre los árboles AST
subestructuras_comunes = comparar_subestructuras_comunes(ast_trees)



In [26]:
def encontrar_subestructuras_comunes(arbol1, arbol2):
    # Función auxiliar para recorrer los árboles AST recursivamente
    def recorrer_arbol(nodo1, nodo2):
        # Si alguno de los nodos es None, no hay subestructuras comunes aquí
        if nodo1 is None or nodo2 is None:
            return []

        # Si los tipos de los nodos son diferentes, no hay subestructuras comunes aquí
        if type(nodo1) != type(nodo2):
            return []

        # Si los tipos de los nodos son iguales, verifica si son del mismo tipo
        if isinstance(nodo1, javalang.ast.Node) and isinstance(nodo2, javalang.ast.Node):
            # Si los nodos son del mismo tipo, compara sus subnodos recursivamente
            subestructuras_comunes = []
            for subnodo1, subnodo2 in zip(nodo1.children, nodo2.children):
                subestructuras_comunes.extend(
                    recorrer_arbol(subnodo1, subnodo2))
            return subestructuras_comunes

        # Si los nodos son de tipo primitivo (como str, int, etc.), compara sus valores
        if nodo1 == nodo2:
            # Si los valores son iguales, se ha encontrado una subestructura común
            return [(nodo1, nodo2)]
        else:
            # Si los valores son diferentes, no hay subestructuras comunes aquí
            return []

    # Llama a la función auxiliar para iniciar el recorrido recursivo de los árboles
    return recorrer_arbol(arbol1, arbol2)


# Comparación de subestructuras comunes entre los árboles AST almacenados en un diccionario
def comparar_subestructuras_comunes(ast_trees):
    # Diccionario para almacenar las subestructuras comunes encontradas en los árboles
    subestructuras_comunes = {}

    # Recorrer cada par de árboles AST para compararlos
    for nombre_archivo1, arbol1 in ast_trees.items():
        for nombre_archivo2, arbol2 in ast_trees.items():
            # Evitar comparar un árbol consigo mismo
            if nombre_archivo1 != nombre_archivo2:
                # Comparar los árboles AST y encontrar las subestructuras comunes
                subestructuras = encontrar_subestructuras_comunes(
                    arbol1, arbol2)

                # Si hay subestructuras comunes, guardar la información
                if subestructuras:
                    # Almacenar el nombre de los archivos equivalentes en un conjunto
                    if nombre_archivo1 not in subestructuras_comunes:
                        subestructuras_comunes[nombre_archivo1] = set()
                    subestructuras_comunes[nombre_archivo1].add(
                        nombre_archivo2)

    return subestructuras_comunes

# Función para imprimir los programas estructuralmente equivalentes


# Función para imprimir los programas estructuralmente equivalentes con puntaje de similitud
def imprimir_programas_equivalentes(subestructuras_comunes):
    print("Programas estructuralmente equivalentes (ordenados por similitud):")
    # Lista para almacenar los programas equivalentes con su puntaje de similitud
    programas_similitud = []
    for nombre_archivo, archivos_equivalentes in subestructuras_comunes.items():
        if archivos_equivalentes:
            # Calcular el puntaje de similitud basado en el número de archivos equivalentes
            similitud = len(archivos_equivalentes)
            programas_similitud.append((nombre_archivo, similitud, archivos_equivalentes))
    
    # Ordenar los programas por similitud (de mayor a menor)
    programas_similitud.sort(key=lambda x: x[1], reverse=True)
    
    # Imprimir los programas estructuralmente equivalentes con su puntaje de similitud
    for programa, similitud, equivalentes in programas_similitud:
        print(f"- {programa} (Similitud: {similitud}):")
        for equivalente in equivalentes:
            print(f"  - {equivalente}")


# Llamada a la función para comparar subestructuras comunes entre los árboles AST
subestructuras_comunes = comparar_subestructuras_comunes(ast_trees)

# Llamada a la función para imprimir los programas estructuralmente equivalentes
imprimir_programas_equivalentes(subestructuras_comunes)

Programas estructuralmente equivalentes (ordenados por similitud):
- 115c99cb.java (Similitud: 7):
  - 548ffb07.java
  - 921b6e4a.java
  - 2bbf754b.java
  - 6e207cbf.java
  - 6490bbe8.java
  - 66e74577.java
  - 3e6def38.java
- 2bbf754b.java (Similitud: 7):
  - 548ffb07.java
  - 921b6e4a.java
  - 6e207cbf.java
  - 6490bbe8.java
  - 66e74577.java
  - 3e6def38.java
  - 115c99cb.java
- 3e6def38.java (Similitud: 7):
  - 548ffb07.java
  - 921b6e4a.java
  - 2bbf754b.java
  - 6e207cbf.java
  - 6490bbe8.java
  - 66e74577.java
  - 115c99cb.java
- 548ffb07.java (Similitud: 7):
  - 921b6e4a.java
  - 2bbf754b.java
  - 6e207cbf.java
  - 6490bbe8.java
  - 66e74577.java
  - 3e6def38.java
  - 115c99cb.java
- 6490bbe8.java (Similitud: 7):
  - 548ffb07.java
  - 921b6e4a.java
  - 2bbf754b.java
  - 6e207cbf.java
  - 66e74577.java
  - 3e6def38.java
  - 115c99cb.java
- 66e74577.java (Similitud: 7):
  - 548ffb07.java
  - 921b6e4a.java
  - 2bbf754b.java
  - 6e207cbf.java
  - 6490bbe8.java
  - 3e6def38.java
  -

In [37]:
import os
import re
import javalang
from munkres import Munkres


def limpiar_archivo_java(archivo):
    with open(archivo, 'r') as file:
        contenido = file.read()

    # Eliminar sangría, saltos de línea y comentarios
    contenido = re.sub(r'//.*|/\*(.|\n)*?\*/', '', contenido)

    # Eliminar líneas de importación de bibliotecas
    contenido = re.sub(r'import\s+.*?;', '', contenido)

    # Eliminar líneas en blanco adicionales
    contenido = re.sub(r'\n\s*\n', '\n', contenido)

    return contenido


def get_significant_subtrees(root):
    """Encuentra los subárboles significativos de un AST.

    Args:
    root: La raíz del AST principal.

    Returns:
    Una lista con todos los subárboles significativos de la raíz.
    """
    def is_significant(node):
        """Determina si un nodo AST es significativo."""
        return (
            isinstance(node, javalang.tree.MethodDeclaration) or
            isinstance(node, javalang.tree.IfStatement) or
            isinstance(node, javalang.tree.ClassDeclaration) or
            isinstance(node, javalang.tree.WhileStatement) or
            isinstance(node, javalang.tree.ForStatement) or
            isinstance(node, javalang.tree.ReturnStatement)
        )

    significant_subtrees = []
    for path, node in root.filter(javalang.tree.Node):
        if is_significant(node):
            significant_subtrees.append(node)
    return significant_subtrees


def bestMatchWeight(bestMatch):
    if isinstance(bestMatch, int):  # Si bestMatch es un solo valor entero
        return bestMatch
    else:
        return sum(bestMatch)



def allSubtreeWeight(subtreesa, subtreesb):
    return len(subtreesa) + len(subtreesb)


def kuhnMunkres(costMatrix):
    m = Munkres()
    indexes = m.compute(costMatrix)
    total_cost = sum(costMatrix[i][j] for i, j in indexes)
    return total_cost


def reorderCompare(ASTa, ASTb, depth, visited):
    def get_node_count(node, visited):
        """Función auxiliar para contar el número de nodos."""
        stack = [(node, ())]
        count = 0
        while stack:
            current_node, path = stack.pop()
            # Usar la tupla (path, type(current_node)) como identificador único del nodo
            node_id = (path, type(current_node))
            if node_id in visited:
                continue
            visited.add(node_id)
            count += 1
            if isinstance(current_node, javalang.tree.Node):
                # Actualizar el path con la ubicación del nodo actual
                new_path = path + (current_node,)
                stack.extend((child, new_path) for child in current_node.children)
        return count



    node_count_a = get_node_count(ASTa, visited)
    node_count_b = get_node_count(ASTb, visited)

    if node_count_a <= 1 or node_count_b <= 1:
        return compareASTs(ASTa, ASTb, depth, visited) + 1
    else:
        costMatrix = [[compareASTs(child_a, child_b, depth, visited)
                       for child_b in ASTb] for child_a in ASTa]
        bestMatch = kuhnMunkres(costMatrix)
        return sum(bestMatch)


def compareASTs(ASTa, ASTb, depth, visited):
    def get_node_count(node, visited):
        """Función auxiliar para contar el número de nodos."""
        stack = [(node, ())]
        count = 0
        while stack:
            current_node, path = stack.pop()
            # Usar la tupla (path, type(current_node)) como identificador único del nodo
            node_id = (path, type(current_node))
            if node_id in visited:
                continue
            visited.add(node_id)
            count += 1
            if isinstance(current_node, javalang.tree.Node):
                # Actualizar el path con la ubicación del nodo actual
                new_path = path + (current_node,)
                stack.extend((child, new_path) for child in current_node.children)
        return count

    node_count_a = get_node_count(ASTa, visited)
    node_count_b = get_node_count(ASTb, visited)

    if isinstance(ASTa, type(ASTb)) and node_count_a == node_count_b == 0:
        return 1
    if not isinstance(ASTa, type(ASTb)) or node_count_a != node_count_b:
        return 0
    if depth == 0:
        return sum(compareASTs(child_a, child_b, depth, visited) for child_a, child_b in zip(ASTa, ASTb)) + 1
    elif depth > 0:
        return reorderCompare(ASTa, ASTb, depth - 1, visited) + 1


def compareSubtrees(subtreesa, subtreesb, depth):
    visited = set()
    node_count_a = len(subtreesa)
    node_count_b = len(subtreesb)

    if node_count_a <= 1 or node_count_b <= 1:
        bestMatch = compareASTs(subtreesa, subtreesb, depth, visited)
    else:
        costMatrix = [[compareASTs(child_a, child_b, depth, visited)
                       for child_b in subtreesb] for child_a in subtreesa]
        bestMatch = kuhnMunkres(costMatrix)

    return 2 * bestMatchWeight(bestMatch) / allSubtreeWeight(subtreesa, subtreesb)


# Comparar los árboles de sintaxis abstracta
carpeta = '.\\conplag\\minor'
archivos_java = [archivo for archivo in os.listdir(
    carpeta) if archivo.endswith('.java')]

ast_trees = {}

for archivo in archivos_java:
    ruta_archivo = os.path.join(carpeta, archivo)
    contenido_limpio = limpiar_archivo_java(ruta_archivo)
    tokens = javalang.tokenizer.tokenize(contenido_limpio)

    try:
        parser = javalang.parser.Parser(tokens)
        ast_tree = parser.parse_member_declaration()
        # Obtener los nodos significativos del árbol AST
        significant_subtrees = get_significant_subtrees(ast_tree)
        # Guardar los nodos significativos en un diccionario con el nombre del archivo como clave
        ast_trees[archivo] = significant_subtrees
    except javalang.parser.JavaSyntaxError as e:
        print("Error de sintaxis en", archivo, ":", e)

for archivo_a, subtrees_a in ast_trees.items():
    for archivo_b, subtrees_b in ast_trees.items():
        if archivo_a != archivo_b:
            similarity = compareSubtrees(subtrees_a, subtrees_b, depth=3)
            print(f"Similitud entre {archivo_a} y {archivo_b}: {similarity}")


Similitud entre 115c99cb.java y 2bbf754b.java: 0.0
Similitud entre 115c99cb.java y 3e6def38.java: 0.0
Similitud entre 115c99cb.java y 548ffb07.java: 0.0
Similitud entre 115c99cb.java y 6490bbe8.java: 0.0
Similitud entre 115c99cb.java y 66e74577.java: 0.0
Similitud entre 115c99cb.java y 6e207cbf.java: 0.0
Similitud entre 115c99cb.java y 921b6e4a.java: 0.0
Similitud entre 2bbf754b.java y 115c99cb.java: 0.0
Similitud entre 2bbf754b.java y 3e6def38.java: 0.0
Similitud entre 2bbf754b.java y 548ffb07.java: 0.0
Similitud entre 2bbf754b.java y 6490bbe8.java: 0.0
Similitud entre 2bbf754b.java y 66e74577.java: 0.0
Similitud entre 2bbf754b.java y 6e207cbf.java: 0.0
Similitud entre 2bbf754b.java y 921b6e4a.java: 0.0
Similitud entre 3e6def38.java y 115c99cb.java: 0.0
Similitud entre 3e6def38.java y 2bbf754b.java: 0.0
Similitud entre 3e6def38.java y 548ffb07.java: 0.0
Similitud entre 3e6def38.java y 6490bbe8.java: 0.0
Similitud entre 3e6def38.java y 66e74577.java: 0.0
Similitud entre 3e6def38.java y